In [47]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
    )

In [107]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 0.01 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    np.random.seed()

    for _ in range(200):
         pos = np.random.random(3) * 9.999 - 4.999
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
    d = datetime.datetime.now()
    time=str(getattr(d, 'year','month'))+str(getattr(d,'month'))+str(getattr(d,'day'))+str(getattr(d,'hour'))+str(getattr(d,'minute'))+str(getattr(d,'second'))
    numbers_a_name = time+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = time+"numbers_b_test1.h5"
    mean_e_name = time+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(1000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc


In [108]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt(t,boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreaction)+np.tanh(reactiondistance*np.sqrt(intrinsikreaction/(Diffusion_E+Diffusion_S))))
    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    c=fa['particleNumbers'][2]*(np.exp((-t*k1*1)/(boxsize**3)))
    c11=fa['particleNumbers'][0]*np.exp((-t*k11*1)/(boxsize**3))

    return t, c,c11

In [120]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [1.5,2.,3.5,5.]:
    boxsize=10.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=3
    intrinsikreaction=10000000000000000000000
    colornow=next(colors)
    falist=[]
    averagelist=[]
    for ii in range(5):
        fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
        #plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
        falist.append(fa['particleNumbers'])
        t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
        averagelist.append(c)
    averagelist=np.array(averagelist) 
    analytnumer=averagelist.mean(axis=0)
    falist=np.array(falist)  
    t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    paertnum=falist.mean(axis=0)   
    plt.plot(fa['times'], paertnum,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(t,c,"--",color=colornow)
plt.ylabel('S(t)', fontsize=10)
plt.xlabel('t in ns', fontsize=10)
plt.legend()

2016/05/21 10:43:50 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:43:55 INFO: Finished after 5.52646 seconds.
2016/05/21 10:43:55 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:01 INFO: Finished after 5.40074 seconds.
2016/05/21 10:44:01 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:06 INFO: Finished after 5.404469 seconds.
2016/05/21 10:44:06 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:11 INFO: Finished after 5.471248 seconds.
2016/05/21 10:44:11 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:18 INFO: Finished after 6.152881 seconds.
2016/05/21 10:44:18 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:24 INFO: Finished after 5.446987 seconds.
2016/05/21 10:44:24 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:29 INFO: Finished after 5.417738 seconds.
2016/05/21 10:44:29 INFO: Run with timestep 0.01 and 1000 timesteps
2016/05/21 10:44:35 INFO: Finished after 5.42

In [121]:
plt.show()

In [104]:
help(rdy)

Help on module revreaddy:

NAME
    revreaddy - A particle-based reaction-diffusion simulation.

FILE
    /home/mi/janekg89/Dokumente/Masterarbeit/revreaddy/build/Release/revreaddy.py

DESCRIPTION
    This module is the convenience/user layer of the underlying
    C++ revreaddy library (C-extension). It offers a simple 
    interface and useful methods to run reaction-diffusion
    simulations, save their results in compact binary files
    and save or load earlier states of a simulation with ease.
    
    It defines the main class Sim, which communicates with
    the C-extension, and methods that act on Sim that are
    considered 'utilities'.
    
    Dependencies are:
    * revreaddyPy (C-extension)
    * numpy
    * h5py

CLASSES
    __builtin__.object
        Sim
    
    class Sim(__builtin__.object)
     |  User interface class to setup and run simulations.
     |  
     |  Main object that wraps the three most important objects
     |  of the C-extension: Config, World and Sim